In [5]:
import os
import re
import glob
import numpy as np
import pandas as pd
from tqdm import tqdm
from pyace import PyACECalculator

# =========================
# Config
# =========================
TEST_PKL = "./Testset/W-test-npj.pckl.gzip"
POT_DIR = "./potentials"
DATASET_IDS = range(0, 6)   # 0-5
REPLICA_IDS = range(0, 5)   # 0-4

# If you want to shift energy, set to True and use mean error shift
USE_ENERGY_SHIFT = False

# =========================
# Load test data
# =========================
df1 = pd.read_pickle(TEST_PKL, compression="gzip")

# True forces (flattened)
force_true = np.vstack(df1["forces"]).flatten()

# True total energies stored in df (often shape (N,1) or (N,))
# We'll compute per-atom energy true: E_total / N_atoms
energy_total_true = np.array(df1["energy"]).reshape(-1)  # safe reshape to 1D

atoms_list = df1["ase_atoms"].tolist()
n_atoms = np.array([a.get_global_number_of_atoms() for a in atoms_list], dtype=int)
energy_peratom_true = (energy_total_true / n_atoms).reshape(-1, 1)

# =========================
# Helper: evaluate one model
# =========================
def evaluate_yaml(yaml_path, atoms_list, n_atoms, energy_peratom_true, force_true):
    calc = PyACECalculator(yaml_path)

    energy_peratom_pred = []
    force_pred_list = []

    # loop over frames
    for a in tqdm(atoms_list, desc=f"Eval {os.path.basename(yaml_path)}", leave=False):
        nat = a.get_global_number_of_atoms()
        a.calc = calc

        e_pa = a.get_potential_energy() / nat
        f = a.get_forces()

        energy_peratom_pred.append(e_pa)
        force_pred_list.append(f)

    energy_peratom_pred = np.array(energy_peratom_pred, dtype=float).reshape(-1, 1)

    # optional energy shift
    if USE_ENERGY_SHIFT:
        shift = float(np.mean(energy_peratom_pred - energy_peratom_true))
    else:
        shift = 0.0

    # energy metrics (per atom)
    e_diff = (energy_peratom_true - (energy_peratom_pred - shift))
    e_mae = float(np.mean(np.abs(e_diff)))
    e_rmse = float(np.sqrt(np.mean(e_diff ** 2)))

    # force metrics (flatten all components)
    force_pred = np.concatenate([np.asarray(f) for f in force_pred_list], axis=0).flatten()
    f_diff = (force_true - force_pred)
    f_mae = float(np.mean(np.abs(f_diff)))
    f_rmse = float(np.sqrt(np.mean(f_diff ** 2)))

    return e_rmse, e_mae, f_rmse, f_mae, shift

# =========================
# Main loop: scan and eval
# =========================
results = []

pattern = re.compile(r"^(?P<ds>[0-5])_ensemble_potential_(?P<rep>[0-4])\.yaml$")

for ds in DATASET_IDS:
    for rep in REPLICA_IDS:
        yaml_path = os.path.join(POT_DIR, f"{ds}_ensemble_potential_{rep}.yaml")
        if not os.path.isfile(yaml_path):
            results.append({
                "dataset": ds,
                "rep": rep,
                "yaml": yaml_path,
                "status": "missing",
                "error": "file not found"
            })
            continue

        print(f"\nProcessing dataset={ds}, rep={rep}: {yaml_path}")
        try:
            e_rmse, e_mae, f_rmse, f_mae, shift = evaluate_yaml(
                yaml_path=yaml_path,
                atoms_list=atoms_list,
                n_atoms=n_atoms,
                energy_peratom_true=energy_peratom_true,
                force_true=force_true
            )

            results.append({
                "dataset": ds,
                "rep": rep,
                "yaml": yaml_path,
                "status": "ok",
                "pe_rmse": e_rmse,
                "pe_mae": e_mae,
                "force_rmse": f_rmse,
                "force_mae": f_mae,
                "shift_magnitude": shift
            })

            print(f"  PE RMSE (eV/atom): {e_rmse:.6f}")
            print(f"  Force RMSE (eV/Å): {f_rmse:.6f}")

        except Exception as e:
            results.append({
                "dataset": ds,
                "rep": rep,
                "yaml": yaml_path,
                "status": "error",
                "error": str(e)
            })
            print(f"Error processing dataset={ds}, rep={rep}: {e}")

# =========================
# Results DataFrame
# =========================
results_df = pd.DataFrame(results)

# Save per-model results (.dat)
# Using tab-separated for easy reading in Linux/Excel
results_df.to_csv("./Testset/results_per_model.dat", sep="\t", index=False)

# =========================
# Summary by dataset (mean & variance)
# =========================
ok_df = results_df[results_df["status"] == "ok"].copy()

summary = (
    ok_df
    .groupby("dataset")
    .agg(
        pe_rmse_mean=("pe_rmse", "mean"),
        pe_rmse_var=("pe_rmse", "var"),
        force_rmse_mean=("force_rmse", "mean"),
        force_rmse_var=("force_rmse", "var"),
        pe_mae_mean=("pe_mae", "mean"),
        pe_mae_var=("pe_mae", "var"),
        force_mae_mean=("force_mae", "mean"),
        force_mae_var=("force_mae", "var"),
        n_models=("pe_rmse", "count")
    )
    .reset_index()
)

print("\nSummary by dataset (mean/var):")
# nicer printing
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    print(summary)

# Save summary (.dat)
summary.to_csv("./Testset/results_summary_by_dataset.dat", sep="\t", index=False)

print("\nSaved:")
print("  - results_per_model.dat")
print("  - results_summary_by_dataset.dat")



Processing dataset=0, rep=0: ./potentials/0_ensemble_potential_0.yaml


  PE RMSE (eV/atom): 0.006443
  Force RMSE (eV/Å): 0.167639

Processing dataset=0, rep=1: ./potentials/0_ensemble_potential_1.yaml


  PE RMSE (eV/atom): 0.006662
  Force RMSE (eV/Å): 0.167516

Processing dataset=0, rep=2: ./potentials/0_ensemble_potential_2.yaml


  PE RMSE (eV/atom): 0.007506
  Force RMSE (eV/Å): 0.167194

Processing dataset=0, rep=3: ./potentials/0_ensemble_potential_3.yaml


  PE RMSE (eV/atom): 0.005812
  Force RMSE (eV/Å): 0.164662

Processing dataset=0, rep=4: ./potentials/0_ensemble_potential_4.yaml


...

  PE RMSE (eV/atom): 0.105923
  Force RMSE (eV/Å): 1.714586

Processing dataset=5, rep=1: ./potentials/5_ensemble_potential_1.yaml


  PE RMSE (eV/atom): 0.098285
  Force RMSE (eV/Å): 1.398529

Processing dataset=5, rep=2: ./potentials/5_ensemble_potential_2.yaml


  PE RMSE (eV/atom): 0.196397
  Force RMSE (eV/Å): 1.556342

Processing dataset=5, rep=3: ./potentials/5_ensemble_potential_3.yaml


  PE RMSE (eV/atom): 0.147796
  Force RMSE (eV/Å): 1.604256

Processing dataset=5, rep=4: ./potentials/5_ensemble_potential_4.yaml


  PE RMSE (eV/atom): 0.021551
  Force RMSE (eV/Å): 0.363713

Summary by dataset (mean/var):
   dataset  pe_rmse_mean   pe_rmse_var  force_rmse_mean  force_rmse_var  \
0        0      0.006716  4.275687e-07         0.166265        0.000003   
1        1      0.005999  1.194462e-07         0.164843        0.000003   
2        2      0.005686  8.021015e-09         0.165044        0.000001   
3        3      0.005697  3.062827e-08         0.165186        0.000002   
4        4      0.006608  2.199513e-07         0.173481        0.000005   
5        5      0.113990  4.197600e-03         1.327485        0.303182   

   pe_mae_mean    pe_mae_var  force_mae_mean  force_mae_var  n_models  
0     0.005300  1.504751e-07        0.117514   1.273506e-06         5  
1     0.004838  2.689688e-08        0.117051   1.229964e-06         5  
2     0.004543  6.494746e-09        0.117090   3.857089e-07         5  
3     0.004642  1.620791e-08        0.116933   8.221495e-07         5  
4     0.005232  6.3554